<a href="https://colab.research.google.com/github/RehanFastians/FakeNewsDetector/blob/main/FakeNewsTesting(CLI%26GUI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install -q streamlit pyngrok transformers requests python-dotenv

In [21]:
%%writefile .env
GNEWS_API_KEY="X"
NGROK_AUTH_TOKEN = "X"


Writing .env


In [22]:
%%writefile app.py
import streamlit as st
import requests
from transformers import pipeline
# Setting up API keys
from dotenv import load_dotenv
load_dotenv()
import os
GNEWS_API_KEY = os.getenv("GNEWS_API_KEY")
GNEWS_URL = "https://gnews.io/api/v4/search"

# Setting up model
@st.cache_resource
def load_model():
    return pipeline(
        "text-classification",
        model="jy46604790/Fake-News-Bert-Detect",
        tokenizer="jy46604790/Fake-News-Bert-Detect"
    )

classifier = load_model()

# function to search queries through GNews API
def search_gnews(statement):
    params = {'q': statement, 'token': GNEWS_API_KEY, 'lang': 'en', 'max': 5}
    try:
        res = requests.get("https://gnews.io/api/v4/search", params=params)
        return res.json().get("articles", [])
    except Exception as e:
        return []

# Web Based User Interface
st.title("📰 Fake News Detector")
st.markdown("Enter a news statement to classify it as **REAL** or **FAKE** with confidence, and see real-time news support.")

user_input = st.text_input("🔎 Enter a news claim:")

if user_input:
    result = classifier(user_input[:500])[0]
    label = "REAL" if result["label"] == "LABEL_1" else "FAKE"
    confidence = round(result["score"], 2)

    st.subheader("📊 Prediction by Model")
    st.write(f"**Label**: `{label}`")
    st.write(f"**Confidence**: `{confidence*100}%`")

    # Fetch News through GNews API (searches trusted news sources)
    articles = search_gnews(user_input)
    if articles:
        st.subheader("📰 Related News Articles")
        for article in articles:
            st.markdown(f"- [{article['title']}]({article['url']})")
    else:
        st.warning("No related news found.")

    # Verdict a/c to fetched result
    st.subheader("🧠 Verdict by internet surfing")
    if label == "FAKE" and articles:
        st.info("⚠️ Likely FAKE, but some similar news found. Cross-check.")
    elif label == "FAKE":
        st.error("❌ FAKE — no supporting real news found.")
    elif label == "REAL" and confidence > 0.9:
        st.success("✅ REAL — high confidence.")
    else:
        st.info("✅ Possibly REAL — verify using relevant sources.")


Overwriting app.py


In [25]:
from pyngrok import ngrok, conf

ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))
# Kill previous tunnels cause its a free version so only one channel is allowed
ngrok.kill()

# Starting tunnel for streamlit
public_url = ngrok.connect(addr="8501", proto="http")

print("🔗 Your Streamlit app is live at:", public_url)

# Launching application
!streamlit run app.py &> /dev/null &

🔗 Your Streamlit app is live at: NgrokTunnel: "https://1d742382a898.ngrok-free.app" -> "http://localhost:8501"
